<a href="https://colab.research.google.com/github/eunShim/BigDataCapstone/blob/main/bigdatacapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import zipfile

zip_path = "/content/EdNet-KT3.zip"  # 로컬 경로

with zipfile.ZipFile(zip_path, 'r') as z:
    file_list = z.namelist()
    print("압축 안에 파일 개수:", len(file_list))
    print("앞 10개:", file_list[:10])

압축 안에 파일 개수: 297917
앞 10개: ['KT3/', '__MACOSX/._KT3', 'KT3/u533092.csv', 'KT3/u5901.csv', 'KT3/u734989.csv', 'KT3/u763609.csv', 'KT3/u347548.csv', 'KT3/u265925.csv', 'KT3/u319352.csv', 'KT3/u406983.csv']


In [34]:
import pandas as pd

with zipfile.ZipFile(zip_path, 'r') as z:
    csv_files = [f for f in z.namelist() if f.endswith(".csv")]
    print("CSV 파일 수:", len(csv_files))

    # 첫 번째 CSV만 예시로 열기
    with z.open(csv_files[0]) as f:
        df = pd.read_csv(f, low_memory=False)

print(df.iloc[:, :5].head())


CSV 파일 수: 297915
       timestamp action_type item_id   source user_answer
0  1552454260435       enter   e2575  my_note         NaN
1  1552454264737        quit   e2575  my_note         NaN
2  1552454266783       enter   e2482  my_note         NaN
3  1552454283275        quit   e2482  my_note         NaN
4  1552470265444       enter   e2482  my_note         NaN


In [35]:
import zipfile
import random
import pandas as pd

zip_path = "/content/EdNet-KT3.zip"

with zipfile.ZipFile(zip_path, 'r') as z:
    # 1. 유저별 CSV 파일 경로 가져오기
    user_files = [f for f in z.namelist() if f.startswith("KT3/") and f.endswith(".csv")]
    print("총 사용자 파일 수:", len(user_files))

    # 2. 500명 랜덤 추출 (총 수가 500명 미만이면 전부 추출)
    k = min(500, len(user_files))
    sample_files = random.sample(user_files, k)
    print("샘플 사용자 파일 예시:", sample_files[:5])

    # 3. 500명의 데이터를 하나의 DataFrame으로 합치기
    dfs = []
    for f in sample_files:
        with z.open(f) as fp:
            df = pd.read_csv(fp, low_memory=False)
            df["source_file"] = f.split("/")[-1].replace(".csv", "")  # 유저ID 기록
            dfs.append(df)

    df_sample = pd.concat(dfs, ignore_index=True)

print("샘플 데이터 크기:", df_sample.shape)
print(df_sample.head())


총 사용자 파일 수: 297915
샘플 사용자 파일 예시: ['KT3/u284978.csv', 'KT3/u814998.csv', 'KT3/u338032.csv', 'KT3/u437564.csv', 'KT3/u637271.csv']
샘플 데이터 크기: (159196, 7)
       timestamp action_type item_id     source user_answer platform  \
0  1539990579078       enter   b3213  diagnosis         NaN      web   
1  1539990588793      submit   b3213  diagnosis         NaN      web   
2  1539990588861       enter   b3213  diagnosis         NaN      web   
3  1539990594411     respond   q4681  diagnosis           b      web   
4  1539990596961      submit   b3213  diagnosis         NaN      web   

  source_file  
0     u284978  
1     u284978  
2     u284978  
3     u284978  
4     u284978  


In [37]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 풀이 개수: submit 이벤트 개수 = 푼 문제 수
# 소요 시간: enter → submit 사이 timestamp 차이 평균

# df_sample 은 여러 사용자 합친 DataFrame (source_file = 사용자)

# 1. timestamp를 datetime 변환
df_sample["timestamp"] = pd.to_datetime(df_sample["timestamp"])

# 2. 사용자별 특징 추출
user_features = []

for uid, group in df_sample.groupby("source_file"):
    group = group.sort_values("timestamp")

    # enter와 submit 시각 쌍 찾기
    enters = group[group["action_type"] == "enter"]["timestamp"].tolist()
    submits = group[group["action_type"] == "submit"]["timestamp"].tolist()

    times = []
    for i in range(min(len(enters), len(submits))):
        delta = (submits[i] - enters[i]).total_seconds()
        if delta > 0:
            times.append(delta)

    total_attempts = len(submits)
    avg_elapsed = sum(times)/len(times) if times else 0

    # 응답률: 답안을 제출했는지 여부 (user_answer 결측치 여부)
    submitted = group[group["action_type"] == "submit"]
    if len(submitted) > 0:
        answer_rate = submitted["user_answer"].notna().mean()
    else:
        answer_rate = 0

    user_features.append({
        "source_file": uid,
        "total_attempts": total_attempts,
        "avg_elapsed": avg_elapsed,
        "answer_rate": answer_rate
    })

user_stats = pd.DataFrame(user_features)
print(user_stats.head())

# 3. KMeans 군집화
X = user_stats[["total_attempts", "avg_elapsed", "answer_rate"]]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
user_stats["cluster"] = kmeans.fit_predict(X_scaled)

print(user_stats.head())

# 4. 클러스터별 평균 특징
print(user_stats.groupby("cluster").mean(numeric_only=True))


  source_file  total_attempts  avg_elapsed  answer_rate
0      u10233             352     1.485177          0.0
1       u1083              23     0.003791          0.0
2      u13621              22     0.000918          0.0
3      u13667               7     0.000051          0.0
4      u15107             226     1.266452          0.0
  source_file  total_attempts  avg_elapsed  answer_rate  cluster
0      u10233             352     1.485177          0.0        0
1       u1083              23     0.003791          0.0        0
2      u13621              22     0.000918          0.0        0
3      u13667               7     0.000051          0.0        0
4      u15107             226     1.266452          0.0        0
         total_attempts  avg_elapsed  answer_rate
cluster                                          
0             28.909836     0.078647          0.0
1           1196.363636     2.363150          0.0
2            826.000000    12.154969          0.0


In [36]:
!ls -lh /content/drive/MyDrive/EdNet-KT3.zip
!file /content/drive/MyDrive/EdNet-KT3.zip

-rw------- 1 root root 543M Sep 22 04:09 /content/drive/MyDrive/EdNet-KT3.zip
/content/drive/MyDrive/EdNet-KT3.zip: Zip archive data, at least v2.0 to extract, compression method=store
